### Important Note
For this task, we do English -> Obolo (since T5 models exclusively perform English to [other language]) translation; therefore, this has the possibility of harnessing pretrained weights.

## Preprocess Data

In [1]:
model_checkpoint = 't5-small'

### Load Data

In [2]:
# load data
from datasets import load_dataset

train = load_dataset('csv', data_files='../data/train.csv')['train']
test = load_dataset('csv', data_files='../data/test.csv')['train']
VAL_CUTOFF = 500 # random choice for val loss, full test set is 3110, take 500 away for validation only. 
test, val = test.train_test_split(test_size=VAL_CUTOFF).values()

### Create Tokenizers

In [3]:
import json
from dataset import sequential_transforms, tensor_transform
import sys
sys.path.append('../')
from custom_tokenizers.tokenizerv1 import parse_sentence, CustomOboloTokenizer
from transformers import AutoTokenizer

# t5-small tokenizer, based on sentencepiece. this is what we use for the English portion
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# we can either use the above tokenizer, or a custom tokenizer for the Obolo portion. below, we code our custom Obolo tokenizer
obolo_vocab = json.load(open( "../custom_tokenizers/custom_obolo_tokenizer_vocab.json"))

# modify the obolo vocabulary to match the special tokens of the t5-small tokenizer
modified_obolo_vocab = {'<pad>': 0, '</s>': 1, '<unk>': 2}
for token in obolo_vocab:
    if obolo_vocab[token] <= 4: continue
    modified_obolo_vocab[token] = obolo_vocab[token] - 2

obolo_tokenizer = CustomOboloTokenizer(parse_sentence, modified_obolo_vocab, tokenizer.unk_token_id, tokenizer.pad_token_id)

For now, we simply use the original sentencepiece tokenizer + new words, not our custom Obolo tokenizer

In [4]:
print(f"Number of tokens before: {len(tokenizer)}")
tokenizer.add_tokens(list(modified_obolo_vocab.keys())[3:])
print(f"Number of tokens after adding Obolo specific tokens: {len(tokenizer)}")

Number of tokens before: 32100
Number of tokens after adding Obolo specific tokens: 38788


### Preprocess Dataset
Use the desired tokenizer (in this case, sentencepiece + new vocab) to preprocess dataset from strings to input ids.

In [5]:
prefix = "translate English to Obolo: "
source_lang = "English"
target_lang = "Obolo"
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + ex for ex in examples[source_lang]]
    targets = [ex for ex in examples[target_lang]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    labels = tokenizer(text_target=targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train = train.map(preprocess_function, batched=True)
val = val.map(preprocess_function, batched=True)
test = test.map(preprocess_function, batched=True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2610 [00:00<?, ? examples/s]

## Train

In [7]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

# create model and data collator
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
# resize the embeddings
model.resize_token_embeddings(len(tokenizer))
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [8]:
batch_size = 2
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    eval_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

In [9]:
import numpy as np
from datasets import load_metric

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

metric = load_metric("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

/tmp/user/22494/ipykernel_865504/107648680.py:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")
/move/u/willsh/miniconda3/envs/224W/lib/python3.11/site-packages/datasets/load.py:759: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [10]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train,
    eval_dataset=val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [11]:
trainer.train()

Epoch,Training Loss,Validation Loss
